In [216]:
import re
import csv
import string
import numpy as np
import pandas as pd

import gensim
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_tags
from gensim.parsing.preprocessing import strip_numeric
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
#from nltk import PorterStemmer
from stop_words import safe_get_stop_words
from stop_words import get_stop_words
stop_words = get_stop_words('english')

import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud
#-------------------------------------

#from nltk.corpus import brown
#from sklearn.decomposition import LatentDirichletAllocation
#from sklearn.feature_extraction.text import CountVectorizer
#import pyLDAvis.sklearn
# Generate Word Cloud

#--------------------------
#import nltk
#nltk.download('wordnet')
PorterStemmer().stem('complications')
#from gensim import corpora
stemmer = PorterStemmer()
from nltk import everygrams
from nltk import FreqDist

In [159]:
# Eliminate Duplicate Items by Product Category and Product Link 
#toclean = pd.read_csv("C:/Users/moh_e/Desktop/JobOffers_dominant_topic_Topics___.csv", sep=',', encoding="ISO-8859-1")
#deduped = toclean.drop_duplicates(subset=['Jop_Link'], keep='first')
#deduped.to_csv('C:/Users/moh_e/Desktop/JobOffers_dominant_topic_Topics__10k.csv', sep=',', encoding="ISO-8859-1")

# 1- Loading data

In [160]:
data =pd.read_csv("C:/Users/moh_e/Desktop/Nurturing Data Talent_2/DB_10k____.csv",sep=',',encoding = "ISO-8859-1")
data.head()

,Job_Title,Job_Skills,Job_Link,Job_Description
0,"Data Scientist (NiFi, DHF)","Bachelor's Degree , SQL , Time Management , No...",https://www.simplyhired.com/job/Tq4KqCrotc06Fp...,EducationBachelor's DegreeSkillsSQLTime Manage...
1,Data Scientist - Defense & Intelligence,"Bachelor's Degree , NoSQL , SQL , 401(k) , Had...",https://www.simplyhired.com/job/OF7HNEC0sGEgBA...,EducationBachelor's DegreeSkillsNoSQLSQL401(k)...
2,Fall 2019/Winter 2020 Applied Science Machine ...,"Doctoral Degree , Master's Degree , Natural La...",https://www.simplyhired.com/job/WaFb8rbhCLxehi...,EducationDoctoral DegreeMaster's DegreeSkillsN...
3,Applied Scientist Fall/Winter 2019 Internship ...,"Doctoral Degree , Master's Degree , Natural La...",https://www.simplyhired.com/job/BL2wIHWVfwAAgw...,EducationDoctoral DegreeMaster's DegreeSkillsN...
4,"Data Scientist/Engineer (HDF, NiFi)","Bachelor's Degree , Master's Degree , Doctoral...",https://www.simplyhired.com/job/1H_ga33Kqka5Ot...,EducationBachelor's DegreeMaster's DegreeDocto...


In [161]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10690 entries, 0 to 10689
Data columns (total 4 columns):
Job_Title          10690 non-null object
Job_Skills         10314 non-null object
Job_Link           10690 non-null object
Job_Description    10690 non-null object
dtypes: object(4)
memory usage: 334.1+ KB


In [162]:
data["Job_Description"][10]

"EducationBachelor's DegreeSkillsNoSQLSQLGitAWSSparkBenefitsFood ProvidedPaid Time OffHealth InsuranceDisability InsuranceGym MembershipElder Research Inc. is a recognized leader in predictive analytics and data science. We pride ourselves in our ability to find creative, cutting edge solutions to real-world problems. We are looking for innovative and inquisitive self-starters who enjoy translating complex models into actionable solutions that deliver real value for our clients.\r\r\r\n\r\r\r\nJob Description\r\r\r\n\r\r\r\nAs a Software Engineer in the Defense & Intelligence Business Unit, you will join a functional team of accomplished Data Scientists and Software Engineers that deliver data science solutions to address the toughest national security challenges. You will work closely with data scientists, business consultants, other software engineers, and the system users to create and deploy applications that help the customer make actionable, data-driven decisions. Candidates shou

# 2- DATA PREPROCESSING 

In [163]:
def CountFreqWords(x):
    mapping =dict.fromkeys(map(ord,string.punctuation))
    words = {}
    for m in x:
        try:
            sent=m.split(' ')
        except:
            pass
        for word in sent:
            word=word.strip()
            word=word.lower()
            word=word.translate(mapping)
            if word in stop_words:
                continue
            if len(word)<3:         
                continue
            if word in words:
                words[word]+=1
            else:
                words[word]=1
    print('done')
    return words

In [164]:
words=CountFreqWords(data["Job_Description"])
df_words = pd.DataFrame(list(words.items()),columns=['word','count'])
TopFreqWords=df_words.nlargest(25, 'count')  # to show 25 max count words
TopFreqWords

done


,word,count
25,data,78294
27,experience,36767
115,business,32247
308,will,26462
208,work,22247
211,team,20970
9,development,18551
145,years,15226
59,management,14363
158,skills,14310


### - List of words not related to our study

In [165]:
# List of stop words to be deleted
df_words =pd.read_csv("C:/Users/moh_e/Desktop/df_words2___.csv",sep=',',encoding = "ISO-8859-1")
df_StopWords=df_words[df_words["dl"]==1]
df_StopWords[1:10]

,Unnamed: 0,word,count,dl
13,150,tools,4608,1.0
19,45,services,3878,1.0
26,49,communication,2704,1.0
35,184,field,2453,1.0
36,53,environment,2427,1.0
41,134,quality,2281,1.0
45,202,performance,2230,1.0
46,80,complex,2118,1.0
50,46,architecture,1981,1.0


In [166]:
# add new stop words
stopwords_=df_StopWords["word"].values.tolist()

### - Creat a list of jobs not related to our study, to be deleted 

In [167]:
keywordsList=["big data","data science","business intelligence","data mining","machine learning","analytics","analyst","data analysis",
              "scientist","business","data engineer","marketing","market","java","python","database engineer","sql","data entry"," data modeling",
              "bi developer","statistical software","hadoop","natural language processing","nlp","finance","big data engineer",
              "data scientist","business intelligence analyst","analyzing"]

In [168]:
def SearchBykeywords(text_):
    Results_list = []
    s =text_
    for j in keywordsList:
        SearchResult=re.findall(j,s)
        if SearchResult!=[] :
            Results_list.append(SearchResult[0])
        else:
            Results_list.append("")        
    ResultText=""
    for i in Results_list:
        ResultText=ResultText +" "+i
        ResultText=strip_multiple_whitespaces(ResultText)
    return ResultText

In [169]:
# Remove the added stopwords
def remove_stopwords(txt):
    query = txt
    querywords = query.split()
    resultwords  = [word for word in querywords if word.lower() not in stopwords_]
    result = ' '.join(resultwords)
    return result

In [170]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
            result.append(lemmatize_stemming(token))
    return result

In [197]:
def  Stop_N_grams(x,minimum_n,max_n):    
    MergedText=""
    for i in x:
        MergedText=MergedText+" "+i
    corpus = MergedText.split()
    fd = FreqDist(everygrams(corpus, minimum_n, max_n))
    StopPhrase=pd.DataFrame(list(fd.items()), columns = ["phrase","count"])
    return StopPhrase

In [172]:
def RemoveStopPhrase(text,NgsList):
    UpdatedText=text
    for sentence in NgsList:
        x=UpdatedText.replace(sentence, ' ')
        UpdatedText=x
    return UpdatedText

In [173]:
Search_data=data["Job_Description"].astype(str).apply(lambda x: x.lower())          # Convert text to lowercase
Search_data=Search_data.apply(SearchBykeywords)
data["Search_keywords"]=Search_data

In [174]:
data[data["Search_keywords"]==' '].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 259 entries, 617 to 10683
Data columns (total 5 columns):
Job_Title          259 non-null object
Job_Skills         248 non-null object
Job_Link           259 non-null object
Job_Description    259 non-null object
Search_keywords    259 non-null object
dtypes: object(5)
memory usage: 12.1+ KB


In [175]:
data=data[data["Search_keywords"]!=' ']
#data.info()

In [176]:
data["Job_Description"]=data["Job_Description"].astype(str).apply(lambda x: x.lower())
data["Job_Description"]=data["Job_Description"].apply(strip_multiple_whitespaces) 
data["Job_Description"]=data["Job_Description"].apply(lambda x: x.strip())
data["Job_Description"]=data["Job_Description"].apply(lambda x: re.sub(r"\d+", "", x))

In [177]:
data["Job_Description"][10]

"educationbachelor's degreeskillsnosqlsqlgitawssparkbenefitsfood providedpaid time offhealth insurancedisability insurancegym membershipelder research inc. is a recognized leader in predictive analytics and data science. we pride ourselves in our ability to find creative, cutting edge solutions to real-world problems. we are looking for innovative and inquisitive self-starters who enjoy translating complex models into actionable solutions that deliver real value for our clients. job description as a software engineer in the defense & intelligence business unit, you will join a functional team of accomplished data scientists and software engineers that deliver data science solutions to address the toughest national security challenges. you will work closely with data scientists, business consultants, other software engineers, and the system users to create and deploy applications that help the customer make actionable, data-driven decisions. candidates should have the ability and the wi

In [178]:
MergedData=""
for i in data["Job_Description"]:
    MergedData=MergedData+" "+i
SentencesList = nltk.tokenize.sent_tokenize(MergedData)
SentencesList

[" educationbachelor's degreeskillssqltime managementnosqliathadoopraytheon is supporting a u.s. government customer on a large mission critical development and sustainment program with a mission to deliver cyber capabilities to address emerging threats.",
 'raytheon cybersecurity and special missions (csm) is seeking a sr. principal data scientist with experience in big data analytics using a variety of statistical, algorithmic, data mining and visualization techniques and tools to analyze data from various platforms and multiple disparate or integrated data sources, both structured and unstructured, and develop analytics products & solutions to meet customer cyber security needs.',
 'experience with marklogic nifi and data hub framework (dhf) for data flow management is highly desirable.',
 'the data scientist must be comfortable working with a wide range of stakeholders and functional teams.',
 'they must have a passion for discovering solutions hidden in large data sets and present

In [179]:
SentList=[]
FreqList=[]
x_list=SentencesList
for k in SentencesList:
    i=0
    if k!=" ":
        for j in x_list:
            if j==k :
                i=i+1
        SentList.append(k)
        FreqList.append(i)
dataFreq = {'phrase':SentList,'count':FreqList}
FreqDf=pd.DataFrame(dataFreq)

In [180]:
DuplicateSentencesDf = FreqDf[FreqDf['count'] >= 5].sort_values(by='count', ascending=False)
DuplicateSentencesDf = DuplicateSentencesDf[DuplicateSentencesDf['phrase'] != "."]
DuplicateSentencesDf = DuplicateSentencesDf[DuplicateSentencesDf['phrase'] != "?"]
DuplicateSentencesDf

,count,phrase
153680,337,apex will consider qualified applicants with c...
131390,337,apex will consider qualified applicants with c...
151399,337,apex will consider qualified applicants with c...
151395,337,apex will consider qualified applicants with c...
167716,337,apex will consider qualified applicants with c...
164947,337,apex will consider qualified applicants with c...
127572,337,apex will consider qualified applicants with c...
166329,337,apex will consider qualified applicants with c...
166337,337,apex will consider qualified applicants with c...
165216,337,apex will consider qualified applicants with c...


In [181]:
#Ngrams_df.to_csv('C:/Users/moh_e/Desktop/dfNgrams_df_.csv') 

In [182]:
word_data = DuplicateSentencesDf["phrase"]
nltk_tokens = word_data
ordered_tokens = set()
DupliSentsList = []
for word in nltk_tokens:
    if word not in ordered_tokens:
        ordered_tokens.add(word)
        DupliSentsList.append(word)
        
print(len(DupliSentsList))   

3330


In [183]:
DupliSentsList

['apex will consider qualified applicants with criminal histories in a manner consistent with the requirements of applicable law.',
 'all qualified applicants will receive consideration for employment without regard to race, color, religion, sex, pregnancy, sexual orientation, gender identity, national origin, age, protected veteran status, or disability status.',
 'unisys has more than , employees serving clients around the world.',
 'we do this while protecting and building on their legacy it investments.',
 'our offerings include outsourcing and managed services, systems integration and consulting services, high-end server technology, cybersecurity and cloud management software, and maintenance and support services.',
 'unisys offers a very competitive benefits package including health insurance coverage from first day of employment, a k with an immediately vested company match, vacation and educational benefits.',
 "we work with many of the world's largest companies and government 

In [195]:
def UpdateText(Texts,StopPhrase):
    FullTexts=list(Texts)
    for i in range(len(FullTexts)):
        FullTexts[i]=RemoveStopPhrase(FullTexts[i],StopPhrase)
        
    return FullTexts

In [113]:
data["Job_Description"]=UpdateText(data["Job_Description"],DupliSentsList) 

In [186]:
data["Job_Description"][10]

"educationbachelor's degreeskillsnosqlsqlgitawssparkbenefitsfood providedpaid time offhealth insurancedisability insurancegym membershipelder research inc. is a recognized leader in predictive analytics and data science.     job description as a software engineer in the defense & intelligence business unit, you will join a functional team of accomplished data scientists and software engineers that deliver data science solutions to address the toughest national security challenges. you will work closely with data scientists, business consultants, other software engineers, and the system users to create and deploy applications that help the customer make actionable, data-driven decisions. candidates should have the ability and the willingness to tailor applications to a clientâ\x80\x99s specific business goals using an iterative methodology. required qualifications basic qualifications: + years of software development experience and a ba/bs degree in computer science or related fieldsign

In [115]:
#data.to_csv('C:/Users/moh_e/Desktop/dfdataafterNgrams_df_.csv') 

In [187]:
cleaned_data=data["Job_Description"].astype(str).apply(lambda x: x.lower())                      # Convert text to lowercase
cleaned_data=cleaned_data.apply(strip_non_alphanum)                         # Remove strip_non_alphanum 
cleaned_data=cleaned_data.apply(lambda x: re.sub(r"\d+", "", x))           # Remove numbers 
cleaned_data=cleaned_data.apply(strip_multiple_whitespaces)                                      # Remove whitespaces
cleaned_data=cleaned_data.apply(strip_punctuation)                          # Remove punctuation   
cleaned_data=cleaned_data.apply(lambda x: strip_short(x,minsize=3))         # Remove short words
cleaned_data=cleaned_data.apply(strip_tags)                                 # Remove tags
data["Job_Description"]=cleaned_data
data["Job_Description"][10]

'educationbachelor degreeskillsnosqlsqlgitawssparkbenefitsfood providedpaid time offhealth insurancedisability insurancegym membershipelder research inc recognized leader predictive analytics and data science job description software engineer the defense intelligence business unit you will join functional team accomplished data scientists and software engineers that deliver data science solutions address the toughest national security challenges you will work closely with data scientists business consultants other software engineers and the system users create and deploy applications that help the customer make actionable data driven decisions candidates should have the ability and the willingness tailor applications clientâ specific business goals using iterative methodology required qualifications basic qualifications years software development experience and degree computer science related fieldsignificant experience and expertise programming java python scala javascriptstrong engli

In [188]:
data["Job_Description"]=data["Job_Description"].apply(remove_stopwords)                        # Remove the added stopwords
data["Job_Description"][10]

'inc leader predictive analytics and data science software the intelligence business you team data scientists and software that data science solutions the security you with data scientists business other software and the and that the customer data decisions should the and the business methodology software development and computer science and expertise programming java python and software with some the css aws nosql databases mongodb hadoop spark with data science and software spss sas with and learning algorithms and with databases oracle sql and etl expertise intelligence data and with data and the intelligence such our and data and our through our plan'

In [118]:
# List of words to be deleted
#df_words =pd.read_csv("C:/Users/moh_e/Desktop/df_words3___.csv",sep=',',encoding = "ISO-8859-1")
#df_remove_words=df_words[df_words["dl"]==1]
#stopwords_=df_remove_words["word"].values.tolist()


#cleaned_data=data["Job_Description"].apply(remove_stopwords)                        # Remove the added stopwords
#data["Job_Description"]=cleaned_data

In [201]:
mn_n=[20,15,10,10,10,10,8]
mx_n=[50,40,35,30,30,30,30]
rpt_n=[5,10,10,10,8,5,10]
UpdatedTexts=data["Job_Description"]
for i in zip(mn_n, mx_n, rpt_n):
    NGRMs_df= Stop_N_grams(UpdatedTexts,i[0],i[1])
    NGRMs_df = NGRMs_df[NGRMs_df['count'] >= i[2]].sort_values(by='count', ascending=False)
    NGRMs_df["phrase"]=NGRMs_df["phrase"].astype(str).apply(strip_punctuation)
    NGRMs_df["phrase"]=NGRMs_df["phrase"].astype(str).apply(strip_multiple_whitespaces)
    NgramsList=NGRMs_df["phrase"]
    print(i,"\n", len(NgramsList))
    UpdatedTexts = UpdateText(UpdatedTexts,NgramsList) 
data["Job_Description"]=UpdatedTexts[10]

(20, 50, 5) 
 48895
(15, 40, 10) 
 59
(10, 35, 10) 
 332
(10, 30, 10) 
 31
(10, 30, 8) 
 992
(10, 30, 5) 
 6108
(8, 30, 10) 
 178


In [204]:
data["Job_Description"][10]

'inc leader predictive analytics and data science software the intelligence business you team data scientists and software that data science solutions the security you with data scientists business other software and the and that the customer data decisions should the and the business methodology software development and computer science and expertise programming java python and software with some the css aws nosql databases mongodb hadoop spark with data science and software spss sas with and learning algorithms and with plan'

In [63]:
#data.to_csv('C:/Users/moh_e/Desktop/data-2020.csv')

### - Delete other duplicate jobs

In [206]:
keywordsList2=["Big+Data","Data+Science","Business+Intelligence","Data+Mining","Machine+Learning","Data+Analytics"]

In [207]:
data["Job_Link___"] = UpdateText(data["Job_Link"],keywordsList2) 

In [211]:
SentList=[]
FreqList=[]

x_list=data["Job_Link___"]
for k in data["Job_Link___"]:
    i=0
    if k!=" ":
        
        for j in x_list:
            if j==k :
                i=i+1
        SentList.append(k)
        FreqList.append(i)
        
dataFreq = {'Link':SentList,'count':FreqList}
FreqDf=pd.DataFrame(dataFreq)

Link_Freq = FreqDf[FreqDf['count'] >= 2].sort_values(by='count', ascending=False)

In [212]:
import nltk
word_data = Link_Freq["Link"]
nltk_tokens = word_data

ordered_tokens = set()
result = []

for word in nltk_tokens:
    if word not in ordered_tokens:
        ordered_tokens.add(word)
        result.append(word)

In [213]:
len(result)

877

In [214]:
# Eliminate Duplicate Items by Product Category and Product Link 
#toclean = data
#deduped = toclean.drop_duplicates(subset=['Job_Link___'], keep='first')
#deduped.to_csv('C:/Users/moh_e/Desktop/JobOf10k.csv', sep=',', encoding="ISO-8859-1")


data2 =pd.read_csv("C:/Users/moh_e/Desktop/JobOf10k.csv",sep=',',encoding = "ISO-8859-1")
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9067 entries, 0 to 9066
Data columns (total 7 columns):
Unnamed: 0         9067 non-null int64
Job_Title          9067 non-null object
Job_Skills         8710 non-null object
Job_Link           9067 non-null object
Job_Description    9063 non-null object
Search_keywords    9067 non-null object
Job_Link___        9067 non-null object
dtypes: int64(1), object(6)
memory usage: 495.9+ KB


In [76]:
#data.to_csv('C:/Users/moh_e/Desktop/datas55.csv') 

In [217]:
processed_data=data2["Job_Description"].astype(str).apply(preprocess)
processed_data[10]

['leader',
 'predict',
 'analyt',
 'data',
 'scienc',
 'softwar',
 'intellig',
 'busi',
 'team',
 'data',
 'scientist',
 'softwar',
 'data',
 'scienc',
 'solut',
 'secur',
 'data',
 'scientist',
 'busi',
 'softwar',
 'custom',
 'data',
 'decis',
 'busi',
 'methodolog',
 'softwar',
 'develop',
 'scienc',
 'expertis',
 'program',
 'java',
 'python',
 'softwar',
 'css',
 'aw',
 'nosql',
 'databas',
 'mongodb',
 'hadoop',
 'spark',
 'data',
 'scienc',
 'softwar',
 'spss',
 'sa',
 'learn',
 'algorithm',
 'databas',
 'plan']

In [218]:
processed_data = list(map(lambda line: list(filter(lambda word: word not in stop_words, line)), processed_data))

# 3- Preparing for LDA

In [219]:
# Create Dictionary
import gensim.corpora as corpora
id2word = corpora.Dictionary(processed_data)

In [220]:
# Create Corpus
corpus = [id2word.doc2bow(text) for text in processed_data]

# 4- LDA model training and results visualization

In [221]:
# LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=100,
                                           alpha='auto',
                                           per_word_topics=True)

In [222]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic)) 

Topic: 0 
Words: 0.261*"market" + 0.131*"sale" + 0.112*"custom" + 0.111*"product" + 0.047*"salesforc" + 0.046*"team" + 0.042*"digit" + 0.036*"price" + 0.030*"strategi" + 0.024*"consum"
Topic: 1 
Words: 0.280*"busi" + 0.097*"analyt" + 0.076*"data" + 0.065*"analyst" + 0.058*"analysi" + 0.030*"solut" + 0.029*"intellig" + 0.028*"develop" + 0.026*"sql" + 0.026*"analyz"
Topic: 2 
Words: 0.138*"manag" + 0.097*"project" + 0.066*"process" + 0.061*"develop" + 0.059*"team" + 0.054*"client" + 0.042*"plan" + 0.027*"busi" + 0.024*"assist" + 0.021*"solut"
Topic: 3 
Words: 0.161*"develop" + 0.088*"softwar" + 0.071*"design" + 0.057*"system" + 0.042*"program" + 0.040*"team" + 0.033*"solut" + 0.031*"product" + 0.030*"integr" + 0.027*"code"
Topic: 4 
Words: 0.299*"data" + 0.074*"analyt" + 0.071*"analysi" + 0.071*"statist" + 0.065*"model" + 0.065*"scienc" + 0.026*"program" + 0.026*"team" + 0.023*"analyz" + 0.019*"develop"
Topic: 5 
Words: 0.317*"data" + 0.113*"sql" + 0.086*"databas" + 0.043*"etl" + 0.042*"

In [223]:
# Compute Perplexity Score
print('\nPerplexity Score: ', lda_model.log_perplexity(corpus))

# Compute Coherence Score
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=processed_data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity Score:  -4.124973554904136

Coherence Score:  0.4527748917357267


In [224]:
num_topics=7

def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 30);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);

In [225]:
Topics=get_lda_topics(lda_model, num_topics)

In [226]:
Topics

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07
0,market,busi,manag,develop,data,data,learn
1,sale,analyt,project,softwar,analyt,sql,machin
2,custom,data,process,design,analysi,databas,algorithm
3,product,analyst,develop,system,statist,etl,python
4,salesforc,analysi,team,program,model,warehous,deep
5,team,solut,client,team,scienc,big,product
6,digit,intellig,plan,solut,program,oracl,problem
7,price,develop,busi,product,team,model,scienc
8,strategi,sql,assist,integr,analyz,design,model
9,consum,analyz,solut,code,develop,integr,artifici


In [227]:
# Visualize the topics
import pyLDAvis
import pyLDAvis.gensim 
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='tsne')
vis


PreparedData(topic_coordinates=            Freq  cluster  topics          x          y
topic                                                  
2      27.016928        1       1 -68.289711 -52.660679
3      21.282961        1       2 -73.109741  21.399546
4      16.239975        1       3  55.166149  29.748131
1      14.184481        1       4  59.986187 -44.311916
5      10.358335        1       5  -6.561772 -11.456267
6       6.210401        1       6 -11.381937  62.604000
0       4.706919        1       7  -1.741707 -85.516472, topic_info=     Category          Freq        Term         Total  loglift  logprob
term                                                                   
3     Default  57484.000000        data  57484.000000  30.0000  30.0000
1     Default  29597.000000        busi  29597.000000  29.0000  29.0000
18    Default  14434.000000       learn  14434.000000  28.0000  28.0000
38    Default  23551.000000       manag  23551.000000  27.0000  27.0000
100   Default   7651.000000      market   7651.000000  26.0000  26.0000
44    Default  16377.000000     project  16377.000000  25.0000  25.0000
29    Default  11636.000000     softwar  11636.000000  24.0000  24.0000
0     Default  16632.000000      analyt  16632.000000  23.0000  23.0000
37    Default   6335.000000      machin   6335.000000  22.0000  22.0000
34    Default  11162.000000     product  11162.000000  21.0000  21.0000
89    Default  10570.000000         sql  10570.000000  20.0000  20.0000
11    Default  36552.000000     develop  36552.000000  19.0000  19.0000
2     Default   9003.000000      custom   9003.000000  18.0000  18.0000
32    Default   7151.000000     statist   7151.000000  17.0000  17.0000
6     Default  14046.000000     analysi  14046.000000  16.0000  16.0000
57    Default   9098.000000      client   9098.000000  15.0000  15.0000
62    Default   3845.000000        sale   3845.000000  14.0000  14.0000
10    Default   8011.000000     databas   8011.000000  13.0000  13.0000
7     Default   5768.000000     analyst   5768.000000  12.0000  12.0000
27    Default   9309.000000      scienc   9309.000000  11.0000  11.0000
23    Default   7145.000000        plan   7145.000000  10.0000  10.0000
20    Default  10242.000000       model  10242.000000   9.0000   9.0000
46    Default  11171.000000      system  11171.000000   8.0000   8.0000
5     Default   2777.000000   algorithm   2777.000000   7.0000   7.0000
42    Default  14008.000000     process  14008.000000   6.0000   6.0000
36    Default  14471.000000      design  14471.000000   5.0000   5.0000
47    Default  20810.000000        team  20810.000000   4.0000   4.0000
72    Default   2805.000000         etl   2805.000000   3.0000   3.0000
43    Default  11581.000000     program  11581.000000   2.0000   2.0000
111   Default   2686.000000    warehous   2686.000000   1.0000   1.0000
...       ...           ...         ...           ...      ...      ...
9      Topic7    430.050641    commerci    903.299262   2.3140  -4.2230
148    Topic7    446.625174    competit   1178.722567   2.0857  -4.1852
2      Topic7   3284.427055      custom   9003.538022   2.0477  -2.1899
34     Topic7   3245.920377     product  11162.370388   1.8210  -2.2017
64     Topic7    865.817628    strategi   3217.748766   1.7434  -3.5232
143    Topic7     91.563411     increas    385.625540   1.6183  -5.7699
113    Topic7     20.968504    discount    100.808771   1.4859  -7.2439
17     Topic7    292.436451      leader   1668.302860   1.3148  -4.6086
169    Topic7     24.635015       smart    143.864829   1.2914  -7.0827
93     Topic7    324.799369     strateg   2318.111444   1.0908  -4.5037
135    Topic7     37.804655         saa    286.004582   1.0326  -6.6545
99     Topic7     82.830071     comfort    698.617554   0.9238  -5.8701
120    Topic7     29.586663       engin    287.727000   0.7814  -6.8996
95     Topic7     85.314772    forecast    855.665752   0.7506  -5.8405
74     Topic7    217.865729       optim   2421.000764   0.6481  -4.90

In [90]:
#pyLDAvis.save_html(vis, 'C:/Users/moh_e/Desktop/Nurturing Data Talent_2/Fainal/Visualize topics (Jobs).html')

In [216]:
#import pandas as pd

#top_words_per_topic = []
#for t in range(lda_model.num_topics):
 #   top_words_per_topic.extend([(t, ) + x for x in lda_model.show_topic(t, topn = 30)])

#pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word', 'P']).to_csv("C:/Users/moh_e/Desktop/top_words(jobs).csv")

## - Finding topics for each document

In [94]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [96]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=processed_data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(20)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,1.0,0.3169,"busi, analyt, data, analyst, analysi, solut, i...","[custom, data, data, analyt, busi, lifecycl]"
1,1,4.0,0.4197,"data, analyt, analysi, statist, model, scienc,...","[leader, predict, analyt, data, scienc, data, ..."
2,2,3.0,0.3475,"develop, softwar, design, system, program, tea...",[product]
3,3,6.0,0.3473,"learn, machin, algorithm, python, deep, produc...","[scienc, machin, learn, math, statist, process..."
4,4,2.0,0.3869,"manag, project, process, develop, team, client...",[lifecycl]
5,5,3.0,0.3474,"develop, softwar, design, system, program, tea...",[product]
6,6,2.0,0.4801,"manag, project, process, develop, team, client...","[organ, technolog]"
7,7,2.0,0.4275,"manag, project, process, develop, team, client...","[manag, analyt, busi, expect]"
8,8,2.0,0.2916,"manag, project, process, develop, team, client...","[data, technolog]"
9,9,4.0,0.4208,"data, analyt, analysi, statist, model, scienc,...","[data, scientist, develop, analyt]"


In [97]:
#df_dominant_topic.to_csv('C:/Users/moh_e/Desktop/JobOffers_dominant_8topics.csv')

## - The most representative document for each topic

In [98]:
# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet.head(10)

,Topic_Num,Topic_Perc_Contrib,Keywords,Representative Text
0,0.0,0.7883,"market, sale, custom, product, salesforc, team, digit, price, strategi, consum","[market, design, market, market, organ, market, big, market, busi, market, dynam, market, market..."
1,1.0,0.7194,"busi, analyt, data, analyst, analysi, solut, intellig, develop, sql, analyz","[client, intellig, analyst, intellig, analyst, custom, intellig, analyst, analysi, intellig, dat..."
2,2.0,0.9627,"manag, project, process, develop, team, client, plan, busi, assist, solut","[project, develop, project, assist, distribut, project, project, busi, project, process, project..."
3,3.0,0.9207,"develop, softwar, design, system, program, team, solut, product, integr, code","[develop, technolog, develop, technolog, problem, team, product, integr, integr, product, develo..."
4,4.0,0.7815,"data, analyt, analysi, statist, model, scienc, program, team, analyz, develop","[scienc, data, predict, data, project, develop, statist, data, decis, benefit, dataset, develop,..."
5,5.0,0.7825,"data, sql, databas, etl, warehous, big, oracl, model, design, integr","[data, client, data, data, data, warehous, design, data, expertis, design, expertis, big, data, ..."
6,6.0,0.7312,"learn, machin, algorithm, python, deep, product, problem, scienc, model, artifici","[salesforc, deep, learn, machin, learn, process, learn, extract, learn, machin, classif, team, s..."
